In [1]:
#This will be used to embed images in Jupyter Notebook

from IPython.display import Image
#While importing this is important to capitalize the I and P in IPython.
#if this is not done then this might be why there is an error


#Importing modules


import sys 
#this is used to manipulate variable and functions. 
#According to online resources this is helpful mostly for run time

import os
#This is useful when dealing with differnent directories in python.
##It can remove, change, or help accese directories

import glob
#This can be helpful when trying to return a file path that follows 
#a specific parttern

from os import listdir
#Shows a file or list of all files and directories that is in a working directory

from os import path
#I am not sure what this one does



#This is for arrays
import numpy as np
import pandas as pd

#For fit files and WCS objects
import astropy.units as u
from astropy.time import Time


#plotting and drawing modules

from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns



#additional libraroes needed for AFINO

from scipy.io import readsav as idlsave

#import stats librarry

import scipy.stats as stats
import scipy.signal as sig

#This is the imports I used for AFINO
from afino import afino_start
from os import listdir
import json
import pandas as pd

In [2]:
def recrate(flx, h): 
    '''compue the difference formula for f'(a) with step size h.
    
    parameters 
    -----------
    flx : array that contains reconnection flux 
    h : contant cadance between observationss
    
    Returns
    ---------
    recreate: reconnection flux(a+h) - reconnection flux(a-h)/2h 
            [negative reconnection rates, posative reconnection rate]
    '''
    
    recreate  = np.zeros((2, len(flx[1,:])))
    
    nflux       = flx[2]
    pflux      = flx[0]
    recreate[0] = np.gradient(nflux, h, edge_order = 2)
    recreate[1] = np.gradient(pflux, h, edge_order = 2) 
    
    return recreate

In [3]:

def check(filename,bstr,wvstr):
    '''
    
    inputs: 
            filename -- name of the reconnection flux idl save file
            bstr     -- 
                        BLOS - normal component taken as the line of sight
                        BRAD - normal component taken as the radial magnatic field (B_r)
            wvstr    -- 
    output: 
            savname  -- full filename with extension of the reconnection flux
    '''
    for i in range(len(bstr)): 
        if path.exists(filename+bstr[i]+wvstr+".sav" ) == True:
            return  filename+bstr[i]+wvstr+".sav"
        else: 
            print('File not found!')
            

In [4]:
def extract_ribbon_vars(idlstrct):
    '''
    Written by Marcel
    Oct. 5, 2020
    Inputs  --
    Outputs --  OPTIONAL (*) 
                *dosday --?
                flx6 -- magnetic reconnection flux for pixel area with I >= median(I)*6
                flx8 -- magnetic reconnection flux for pixel area with I >= median(I)*8
                flx10 -- magnetic reconnection flux for pixel area with I >= median(I)*10
                * area6 --?
                * area8 --? 
                * area10 --?
                * ar_area --? 
                * mflx --?
                tim -- UTC time from HMI 
                * cut --? 
                * sathr --? 
                * str_brad --? 
                * tim_maxnegrecrate --?    
                * tim_maxposrecrate --?       
                * maxnegrecrate --?        
                * maxposrecrate --?         
                n_o_satpix -- number of saturated points for a given AIA image          
                * bcenx6 --?               
                * bceny6 --?             
                * bcenx8 --?            
                * bceny8 --?            
                * bcenx10 --?             
                * bceny10 --?            
                * dst6 --?                
                * dst8 --?               
                * dst10 --?               

                
    
    Extract all the idl variables 
    '''
    
    flx6                = idlstrct.flx6 
    flx8                = idlstrct.flx8
    flx10               = idlstrct.flx10 
    tim                 = list(idlstrct.tim) #object arrays are not suported by python  
    n_o_satpix          = idlstrct.n_o_satpix
    # These have been commented out because they are not used in the qpp analysis 
    dosday              = idlstrct.dosday
    area6               = idlstrct.area6 
    area8               = idlstrct.area8 
    area10              = idlstrct.area10 
    ar_area             = idlstrct.ar_area  
    mflx                = idlstrct.mflx
    cut                 = idlstrct.cut
    satthr              = idlstrct.satthr
    dflxdt              = idlstrct.dflxdt
    str_brad            = idlstrct.str_brad

    tim_maxnegrecrate   = idlstrct.tim_maxnegrecrate 
    tim_maxposrecrate   = idlstrct.tim_maxposrecrate   
    maxnegrecrate       = idlstrct.maxnegrecrate
    maxposrecrate       = idlstrct.maxposrecrate 
    bcenx6              = idlstrct.bcenx6
    bceny6              = idlstrct.bceny6
    bcenx8              = idlstrct.bcenx8
    bceny8              = idlstrct.bceny8
    bcenx10             = idlstrct.bcenx10
    bceny10             = idlstrct.bceny10
    dst6                = idlstrct.dst6
    dst8                = idlstrct.dst8
    dst10               = idlstrct.dst10
    
    return dosday,flx8,flx6,flx10,area6,area8,area10,ar_area,mflx,tim,cut,satthr,dflxdt,str_brad,tim_maxnegrecrate,tim_maxposrecrate,maxnegrecrate,maxposrecrate,n_o_satpix,bcenx6,bceny6,bcenx8,bceny8,bcenx10,bceny10,dst6,dst8,dst10 
    #return dosday,flx8,flx6,flx10,area6,area8,area10,ar_area,mflx,tim,cut,satthr,str_brad,tim_maxnegrecrate,tim_maxposrecrate,maxnegrecrate,maxposrecrate,n_o_satpix,bcenx6,bceny6,bcenx8,bceny8,bcenx10,bceny10,dst6,dst8,dst10 
    #return flx6,flx8,flx10,tim,n_o_satpix,area6,area8,area10,ar_area

In [5]:
def utc2jd(tim): 
    '''
    Written by Marcel 
    Oct. 2020
    
    Imput -- 
                tim -- UTC time from IDL sav file
    Output -- 
                jdt -- Julian days converted from UTC time  
                jst -- Julian days converted into seconds 
                js  -- Seconds from the start of the start date in Julian days
                to  -- Time Object dates in UTC-ISOT format
    '''
    jdt = list(np.zeros(len(tim)))
    jst = list(np.zeros(len(tim)))
    js  = list(np.zeros(len(tim)))
    sts = list(np.zeros(len(tim)))

    for i in range(len(tim)): 
        utcstr = str(tim[i])
        utcstr = utcstr.replace('b',"")
        utcstr = utcstr.replace("'",'') 
        sts[i] = utcstr #array to format date that is read by astropy
        jdt[i] = Time(utcstr,format='isot')
        jdt[i] = jdt[i].jd
        jst[i] = jdt[i] * 24 * 3600
        js[i]  = jst[i] - jst[0]
    jdt = np.array(jdt)
    jst = np.array(jst)
    js  = np.array(js)
    to  = Time(sts,scale='utc',format='isot') 
    return jdt,jst,js,to


In [6]:
def extract_RDB(flare_data):
    #extract all variables from RibbonDB IDL savefiles
    dosday,rdbflx8,rdbflx6,rdbflx10,area6,area8,area10,ar_area,mflx,tim,cut,satthr,dflxdt,str_brad,tim_maxnegrecrate,tim_maxposrecrate,maxnegrecrate,maxposrecrate,n_o_satpix,bcenx6,bceny6,bcenx8,bceny8,bcenx10,bceny10,dst6,dst8,dst10 =extract_ribbon_vars(flare_data)
    #rdbflx6,rdbflx8,rdbflx10,tim,n_o_satpix,area6,area8,area10,ar_area = extract_ribbon_vars(flare_data)
    rdbflx = (rdbflx6+rdbflx10)/2 # reconnection flux in Maxwells
   
    #changes from utc to julian calander (in seconds; see ribbon_functions.py)
    _,_,rdbt,rdbtt = utc2jd(tim) #time is stored in a time stamp from year, month, day, etc...
    
    # Evaluate the reconnection rate
    #   negative recflux = rflx[0,:]    
    #   positive recflux = rflx[1,:]
    rdbrflx  = recrate(rdbflx,rdbt[1]-rdbt[0]) # reconnection rate 
                                               # in units of Maxwells per second
    
    #conversion of time to minutes
    rdbt = rdbt/60
       
    return rdbflx,rdbrflx,rdbt,rdbtt,area8,n_o_satpix,ar_area,maxnegrecrate,maxposrecrate


#(Uses the previously defined functions to convert rdbt data from seconds to minutes)


In [7]:
AFINO_variables = []


In [8]:
import os
names = os.listdir('/Users/veronicaestrada/Downloads/Kazachenko/files/')

In [9]:
short_names = np.zeros(len(names), dtype = 'U30')
for i in range(len(names)):
    short_names[i] = names[i][7:-26]
    print(names[i][7:-26])
    

20110310_0350_11166_C2.9
20131013_0012_11865_M1.7
20141025_1655_12192_X1.0
20141020_1600_12192_M4.5
20150822_0909_12403_C5.1
20131214_1100_11917_C2.3
20151104_0201_12443_C1.1
20120319_2155_11434_C3.5
20110907_2232_11283_X1.8
20141220_0011_12242_X1.8
20140730_1600_12127_C9.0
20141022_1402_12192_X1.6
20111106_1430_11339_C5.3
20120702_1432_11513_C1.7
20140214_0240_11974_M2.3
20120325_0015_11444_C3.0
20141219_0931_12242_M1.3
20150929_2250_12422_C3.7
20141217_2102_12242_C7.1
20140213_1233_11974_C3.0
20110309_2313_11166_X1.5
20120511_0937_11476_C2.3
20150312_1138_12297_M1.6
20111107_0305_11339_C3.5
20110906_2212_11283_X2.1
20120702_1934_11515_C2.5
20131107_1415_11890_M2.4
20111226_2012_11387_M2.3
20140618_0312_12087_C4.0
20131011_2256_11861_C6.3
20120704_0947_11515_M5.3
20110309_2313_11166_X1.5
20120710_1208_11520_C5.5
20131028_1132_11877_M1.4
20110213_1728_11158_M6.6
20120513_0721_11476_C7.0
20120114_0319_11396_C2.1
20101115_0728_11124_C2.3
20120704_0428_11515_M2.3
20140423_0048_12038_C4.3


In [10]:
exts = np.sort(np.array([*set(short_names)]))[1:]

In [11]:
#### Step 2 : Creating new functions 
'''With the help of my amazing Mentor!!! 
Thank you for teaching me important skills'''
plotdir = '/Users/veronicaestrada/Downloads/Kazachenko/plots/'
datadir ="/Users/veronicaestrada/Downloads/Kazachenko/"
rdbdir  = datadir+'files'

for i in range(len(exts)):
    ext = exts[i]


    ##Finds the location of the file and renames it to the date,time, type of flare, and wavelength

    sathr    = [5000.000,4500.000]
    bstr     = ['{:.2f}'.format(sathr[0]) + r"_brad",'{:.2f}'.format(sathr[1]) + r"_brad"]
    wvstr    = r""
    flarename   = check(rdbdir + "/recfile"+ext+r"_cut08_sat",bstr,wvstr) 
    
    print (flarename )
    flare_data  = idlsave(flarename,verbose=0) 
    

    ##what is contained within the variable(flare_data) is dependant on wht I have in my extract_ribbon_vars function. This can all be saved to compare later.

    ####Flux Vs. Time 
    flx8_0 = flare_data.flx8[0] #Posative Flux 
    flx8_2 = flare_data.flx8[2] #Negative Flux

    plt.plot(flx8_0/1e21, label = 'Negative RF',color ='blue' )
    plt.plot(flx8_2/1e21, label= 'Posative RF', color = 'red')
    plt.legend(loc = 'upper right')
    plt.title('Flux Vs. Time',fontsize = 20)
    plt.ylabel('Flux',fontsize = 15)
    plt.xlabel('Time (min)',fontsize = 15)
    plt.legend(fontsize = 15)
    plt.savefig(plotdir+'flux_vs_time/' + ext + '_timeseries.tiff')
    plt.close('all')
    #before exiting plt.close('all')
    
    

    rdbflx,rdbrflx,rdbt,rdbtt,area8,n_o_satpix,ar_area,maxnegrecrate,maxposrecrate = extract_RDB(flare_data)


    ###Posative and negative reconnection rates
    precrate = rdbrflx[0]
    nrecrate = rdbrflx[1]

    ###finds the peaks for the reconnection rate so we can save the Reconnection Rate Vs Time[min]
    posative_peaks = sig.find_peaks(precrate)
    negative_peaks = sig.find_peaks(np.abs(nrecrate))

    peaks_prate = precrate[posative_peaks[0]]
    peaks_ptim = rdbt[posative_peaks[0]]

    peaks_nrate = nrecrate[negative_peaks[0]]
    peaks_ntim = rdbt[negative_peaks[0]]

    ###Plot for Reconnection Rate( Vs. Time (min)

    plt.plot(rdbt,precrate/1e19,color = 'r', alpha = 0.5, linewidth = 2,zorder = 2)
    plt.plot(peaks_ptim,peaks_prate/1e19, 'ro',zorder = 1)


    plt.plot(rdbt,nrecrate/1e19,color = 'b', alpha = 0.5, linewidth = 2,zorder = 2)
    plt.plot(peaks_ntim,peaks_nrate/1e19, 'bo',zorder = 1)
    plt.title('Reconnection Rate Vs. Time Negative',fontsize = 20)
    plt.ylabel('Reconnection Rate ($\\frac{Mx}{s}$)',fontsize = 15)
    plt.xlabel('Time (min)', fontsize =15)
    plt.savefig(plotdir+'timeseries/' + ext + '_timeseries.tiff')
    plt.close('all')
    
    
    #plotting oscilation period
    
    plt.hist(peaks_ptim[1:-1] - peaks_ptim[0:-2],color = 'red',histtype = 'stepfilled',linewidth = 3,alpha = 0.5, label = 'postive')
    plt.hist(peaks_ntim[1:-1] - peaks_ntim[0:-2],color = 'blue',histtype = 'step',linewidth = 3, linestyle = 'dashed',label = 'negative')
    plt.ylabel('Counts [#]',fontsize = 15)
    plt.xlabel('Period [min]',fontsize = 15)
    plt.title('Counting Peaks', fontsize = 20)
    plt.savefig(plotdir+'counting_peaks/' + ext + '_counting_peaks.tiff')
    plt.legend(fontsize = 15)
    
    #Mode to find the period
   
    #f'The period of the negative reconnection rate is {stats.mode(peaks_ntim[1:-1] - peaks_ntim[0:-2])[0][0]} minutes')
    
    #mean to find the period 
    
    def mode(peaks_tim):
        
        if len(stats.mode(peaks_tim[1:-1] - peaks_tim[0:-2])[0]):
            mode = stats.mode(peaks_tim[1:-1] - peaks_tim[0:-2])[0][0]
        else:
            mode = 0   
        return mode
    
    Pmode_Period = mode(peaks_ptim) 
    Nmode_Period = mode(peaks_ntim) 
    Pmean = np.mean(peaks_ptim[1:-1] - peaks_ptim[0:-2])
    Nmean = np.mean(peaks_ntim[1:-1] - peaks_ntim[0:-2])
    
    
    Afino_dir = '/Users/veronicaestrada/afino_repository/saves/'
    #To clean up my laptop
    for f in os.listdir(Afino_dir):
        os.remove(os.path.join(Afino_dir,f))

    ###Running AFINO
    def AFINO_recrate(recrate,rdbt, Afino_dir):
        
       #To get the RibbonDB time into minutes 
        AFINO= afino_start.analyse_series(rdbt*60,recrate)

        #Step 3 (Overwritting the AF_name variable)
        AF_name=listdir(Afino_dir)

        #Step 4 (opens the file)
        file = open(Afino_dir+AF_name[0])

        #Step 5 (loads the file from step 4)
        AF_dic = json.load(file)

        #Step 6 (Easier to work with what is inside of the AF_variable)
        AF_df= pd.DataFrame.from_dict(AF_dic)

        #Step 7 (Seperates the models)
        m0 = AF_df['m0']
        m1 = AF_df['m1']
        m2 = AF_df['m2']

        #Step 8 Calculate difference in the BIC
        dBIC0 = m1['BIC']- m0['BIC']
        dBIC2 = m1['BIC']- m2['BIC']

        #Step 9 Perfered Model between m0 and m1
        if dBIC0 <= -10:
            QPP0 = 1 #There is a QPP
        else:
            QPP0 = 0

            #Step 10 perfered model between m1 and m2
        if dBIC2 <= -10:
            QPP2 = 1 #There is a QPP
        else:
            QPP2 = 0 #There is not a QPP

        #Step 11 Visulalization 
        dt = (rdbt[1]-rdbt[0])*60 # sampling time of the signal. 
        n = len(recrate)
        ### FFT = Normalization Mode "Backwards"
        ### RFFT = Sampling Frequence
        Fcorf = np.fft.rfft(((recrate-np.mean(recrate))/np.mean(recrate))) # The cofficients are complex

        ## Absolute Value 
        Fpwr = np.abs(Fcorf)**2 # We evaluate the complex conjugate which is reffered to as the Power Density


        Ffreq = np.fft.rfftfreq(n,dt) # The frequencies of each given coefficient


        BIC_m0 = m0['BIC']
        BIC_m1 = m1['BIC']
        BIC_m2 = m2['BIC']

        QPP_Period= (1/(np.exp(m1['params'][4])))/60

        rchi2_m0 = m0['rchi2']
        rchi2_m1 = m1['rchi2']
        rchi2_m2 = m2['rchi2']
        
        return QPP0, QPP2, rchi2_m0, rchi2_m1, rchi2_m2,Ffreq,Fpwr, QPP_Period, m0, m1, m2, BIC_m0, BIC_m1, BIC_m2
    
    
    QPP0_P,QPP2_P,rchi2_m0P,rchi2_m1P,rchi2_m2P,Ffreq_P,Fpwr_P,QPP_Period_P, m0P,m1P,m2P,BIC_m0P, BIC_m1P, BIC_m2P = AFINO_recrate(precrate,rdbt,Afino_dir)
    
    QPP0_N,QPP2_N,rchi2_m0N,rchi2_m1N,rchi2_m2N,Ffreq_N,Fpwr_N,QPP_Period_N,m0N,m1N,m2N,BIC_m0N, BIC_m1N, BIC_m2N = AFINO_recrate(nrecrate,rdbt,Afino_dir)
    #####
    ########Make one for the posative and one for the negative###########
    plt.loglog(m0P['frequencies'],(m0P['best_fit_power_spectrum']/np.mean(m0P['best_fit_power_spectrum'])),zorder=1, label = 'm0' )
    plt.loglog(m1P['frequencies'],(m1P['best_fit_power_spectrum']/np.mean(m1P['best_fit_power_spectrum'])),zorder=1, label = 'm1')
    plt.loglog(m2P['frequencies'],(m2P['best_fit_power_spectrum']/np.mean(m2P['best_fit_power_spectrum'])), zorder=1, label = 'm2')
    plt.loglog(Ffreq_P,Fpwr_P/np.mean(Fpwr_P),zorder=0,alpha=0.5, label = 'Rec. Rate')
    plt.ylim((1e-3, 1e2))
    plt.title('Fourier Power Model Fitting Positive', fontsize = 20)
    plt.ylabel('Norm. Fourier Power', fontsize = 15)
    plt.xlabel('Frequencey (${Hz}$)', fontsize = 15)
    plt.legend(fontsize= 15)
    plt.savefig(plotdir+'fourier_power_model/' + ext + '_fourier_power_model_pos.tiff')
    plt.close('all')
    
    plt.loglog(m0N['frequencies'],(m0N['best_fit_power_spectrum']/np.mean(m0N['best_fit_power_spectrum'])),zorder=1, label = 'm0' )
    plt.loglog(m1N['frequencies'],(m1N['best_fit_power_spectrum']/np.mean(m1N['best_fit_power_spectrum'])),zorder=1, label = 'm1')
    plt.loglog(m2N['frequencies'],(m2N['best_fit_power_spectrum']/np.mean(m2N['best_fit_power_spectrum'])), zorder=1, label = 'm2')
    plt.loglog(Ffreq_N,Fpwr_N/np.mean(Fpwr_N),zorder=0,alpha=0.5, label = 'Rec. Rate')
    plt.ylim((1e-3, 1e2))
    plt.title('Fourier Power Model Fitting Negative', fontsize = 20)
    plt.ylabel('Norm. Fourier Power', fontsize = 15)
    plt.xlabel('Frequencey (${Hz}$)', fontsize = 15)
    plt.legend(fontsize= 15)
    plt.savefig(plotdir+'fourier_power_model/' + ext + '_fourier_power_model_neg.tiff')
    plt.close('all')
    
    total_nflux = np.min(flx8_0)
    total_pflux = np.max(flx8_2)
    
    total_parea = np.max(area8[0]) #double check 
    total_narea = np.max(area8[2])
    
    total_ar_parea = ar_area[0] #this might be wrong check with marcel
    total_ar_narea = ar_area[1]
    total_n_o_satpix = np.sum(n_o_satpix)
    
    ## Now I can try to save the spacifics
    current_row = [ext,Pmode_Period,Nmode_Period,Pmean,Nmean, BIC_m0P, BIC_m1P, BIC_m2P,BIC_m0N, BIC_m1N, BIC_m2N,QPP_Period_P,QPP_Period_N,QPP0_P,QPP2_P,QPP0_N,QPP2_N,rchi2_m0P,rchi2_m1P,rchi2_m2P,rchi2_m0N,rchi2_m1N,rchi2_m2N,total_pflux,total_nflux,total_parea,total_narea,total_ar_parea,total_ar_narea,total_n_o_satpix,maxnegrecrate,maxposrecrate]
    
    AFINO_variables.append(current_row)
   

    
    
    
    #before exiting the for loop plt.close('all')
    #export to spider(it will be faster)


/Users/veronicaestrada/Downloads/Kazachenko/files/recfile20100612_0030_11081_M2.0_cut08_sat5000.00_brad.sav


/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:72: RuntimeWarning: invalid value encountered in reciprocal
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:73: RuntimeWarning: invalid value encountered in reciprocal


 
Analysis summary info:
-----------------------------
 
Lnlike m0 (pow_const): 110.85161884550473
Lnlike m1 (pow_const_gauss): 132.38090740238871
Lnlike m2 (bpow_const): 126.17505081222014
 
BIC m0 (pow_const): -214.3254787827816
BIC m1 (pow_const_gauss): -246.31741753420775
BIC m2 (bpow_const): -237.59458380798452
 
rchi2 m0 (pow_const): 1.3197585771275608
prob. m0 (pow_const): 0.21652273334901892
rchi2 m1 (pow_const_gauss): 0.6977551325038147
prob. m1 (pow_const_gauss): 0.7073772436381327
rchi2 m2 (bpow_const): 0.9569092964526776
prob. m2 (bpow_const): 0.4738147577050058
 
-----------------------------
 
 


/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:72: RuntimeWarning: invalid value encountered in reciprocal
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:73: RuntimeWarning: invalid value encountered in reciprocal
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:72: RuntimeWarning: overflow encountered in exp
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:73: RuntimeWarning: overflow encountered in exp
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/scipy/o

 
Analysis summary info:
-----------------------------
 
Lnlike m0 (pow_const): 121.4454377071034
Lnlike m1 (pow_const_gauss): 127.189220399598
Lnlike m2 (bpow_const): 128.30987861220962
 
BIC m0 (pow_const): -235.51311650597893
BIC m1 (pow_const_gauss): -235.93404352862632
BIC m2 (bpow_const): -241.86423940796348
 
rchi2 m0 (pow_const): 1.3210429997514554
prob. m0 (pow_const): 0.21584345302252733
rchi2 m1 (pow_const_gauss): 0.9013492936996349
prob. m1 (pow_const_gauss): 0.5207724312635655
rchi2 m2 (bpow_const): 0.8849363698929107
prob. m2 (bpow_const): 0.5377350405504049
 
-----------------------------
 
 
/Users/veronicaestrada/Downloads/Kazachenko/files/recfile20101111_1554_11123_C4.3_cut08_sat5000.00_brad.sav


...

/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:72: RuntimeWarning: invalid value encountered in reciprocal
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:73: RuntimeWarning: invalid value encountered in reciprocal
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:72: RuntimeWarning: overflow encountered in exp
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:73: RuntimeWarning: overflow encountered in exp


 
Analysis summary info:
-----------------------------
 
Lnlike m0 (pow_const): -10.11314132571034
Lnlike m1 (pow_const_gauss): -3.5782812317255903
Lnlike m2 (bpow_const): -4.504074175730828
 
BIC m0 (pow_const): 26.58239031211657
BIC m1 (pow_const_gauss): 23.04683161519091
BIC m2 (bpow_const): 21.720363672853438
 
rchi2 m0 (pow_const): 1.3312604086577329
prob. m0 (pow_const): 0.24326474622781047
rchi2 m1 (pow_const_gauss): 0.8690177038375753
prob. m1 (pow_const_gauss): 0.49643471925669513
rchi2 m2 (bpow_const): 0.9350629858714422
prob. m2 (bpow_const): 0.45677481698146905
 
-----------------------------
 
 


/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:72: RuntimeWarning: invalid value encountered in reciprocal
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:73: RuntimeWarning: invalid value encountered in reciprocal
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:72: RuntimeWarning: overflow encountered in exp
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/afino-0.5-py3.9.egg/afino/afino_spectral_models.py:73: RuntimeWarning: overflow encountered in exp
/Users/veronicaestrada/opt/anaconda3/lib/python3.9/site-packages/scipy/o

 
Analysis summary info:
-----------------------------
 
Lnlike m0 (pow_const): 24.694221932702362
Lnlike m1 (pow_const_gauss): 25.597710611472547
Lnlike m2 (bpow_const): 26.81831286080394
 
BIC m0 (pow_const): -43.032336204708834
BIC m1 (pow_const_gauss): -35.30515207120536
BIC m2 (bpow_const): -40.9244104002161
 
rchi2 m0 (pow_const): 0.4151369875983572
prob. m0 (pow_const): 0.8550400417710556
rchi2 m1 (pow_const_gauss): 0.42435290072271753
prob. m1 (pow_const_gauss): 0.8228891428658405
rchi2 m2 (bpow_const): 0.36351862193494555
prob. m2 (bpow_const): 0.8737646316828028
 
-----------------------------
 
 
/Users/veronicaestrada/Downloads/Kazachenko/files/recfile20120704_0428_11515_M2.3_cut08_sat5000.00_brad.sav


KeyError: 'dflxdt'

In [ ]:
AFINO_Tabel= pd.DataFrame(AFINO_variables,columns = ['filename','Pmode_Period','Nmode_Period','Pmean','Nmean', 'BIC_m0P', 'BIC_m1P', 'BIC_m2P','BIC_m0N', 'BIC_m1N', 'BIC_m2N','QPP_Period_P','QPP_Period_N','QPP0_P','QPP2_P','QPP0_N','QPP2_N','rchi2_m0P','rchi2_m1P','rchi2_m2P','rchi2_m0N','rchi2_m1N','rchi2_m2N','total_pflux','total_nflux','total_parea','total_narea','total_ar_parea','total_ar_narea','total_n_o_satpix','maxnegrecrate','maxposrecrate'])
AFINO_Tabel.to_csv("AFINO_Tabel.csv",index=False)

In [ ]:
AFINO_Tabel

In [ ]:
#compare mean and mode and AFINO